# Finding the X boson (or not)

10th BCD ISHEP Cargèse School, March 2025

Lecturer: Bruna Pascual (bruna.pascual@cern.ch)

**20 years from now...** The year is 2043, and you are an experimental particle physicist working on ATLAS experiment at the high-luminosity LHC. A brilliant theory colleague of yours called Prof. Ragnie X has just written an incredible paper: she found a theory which can explain dark matter AND resolves the hierarchy problem... just by adding one new boson! (Don’t ask me how she did this, she is the brilliant one, not me!). This “X boson” should have a mass between about 500 and 1000 GeV (her theory cannot predict this accurately), and luckily for you, if it exists it should be observable at the HL-LHC via its decay to photons... so what are you waiting for? Go see if it is there and if her theory is correct!

**Course description:** This is the premise of your coding project, for which we have around 3hrs. You will work in python with simulated datasets to design an analysis which would be sensitive to the existence of this claimed “X boson”. Later this afternoon, you will apply this analysis to a new dataset of “HL-LHC” data, and your objective will be to determine if the “X boson” exists (with which confidence, and what mass and cross-section)... or not (in which case you will set an upper limit)! In this last part, each group will be assumed to live in parallel universes: for each of you, the existence or not of this boson, and its mass and cross-section, will be different and randomly-chosen.

**Course outcomes:** By the end of this challenge you should have developed the following skills:
- Know how to manipulate large datasets with tools such as `pandas Dataframes`;
- Know how to visualise and compare distributions for different populations,
eg with `matplotlib`;
- Know how to use a `jupyter notebook` to analyse data;
- Know how to apply selections to datasets in order to maxmise the signal-to-background ratio;
- Know how to parametrise a distribution using a functional form, and so extract a backgroubd estimate;
- Know how to test a hypothesis using the CLs method and measure observables relating to a signal;


# Setup

Before starting, we need to make sure you have all the datasets available. Let's get them from the git repository.

In [ ]:
!git clone https://github.com/bpascualdias/SearchingTheXBoson.git

Cloning into 'SearchingTheXBoson'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 29 (delta 0), reused 1 (delta 0), pack-reused 27 (from 1)
Receiving objects: 100% (29/29), 162.85 MiB | 20.52 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (17/17), done.


In [ ]:
!ls SearchingTheXBoson/datasets_simulation

background.csv	signal_1000.csv  signal_500.csv  signal_750.csv


Among the downloaded content, you should be able to see a folder called `datasets_simulation` with csv files inside.

# Part 1: Data wrangling

In the data science jargon, *data wrangling* refers to the process of tidying up and getting to grips with the data, such that one can start to analyse it. The purpose of this first session is therefore to discover the format of your datasets, to learn how to read them and make simple plots with matplotlib (or similar).

The theory proposed by Dr. X. suggests that you should look for the *X boson decaying to pairs of photons*. So, you will need collision events with two photons.

To search for the X boson, you have at your disposal three types of dataset:
- simulated signal samples for a few different assumptions of the unknown X-boson mass, where the X decayed to two photons;
- simulated background sample, which represents the processes from the standard model which also give two-photon final states;
- a observed dataset, which contains events with two photons, but for which you do not know which (if any!) originated from an X-boson;

**Some details in our imaginary scenario... (you will need these numbers later)**

- Let's pretend that the X-boson signal samples were generated with some future version of theMadGraph Monte Carlo event generator, with hadronization and showering provided by Pythia.

- Regardless of assumption of the mass of the X-boson, the **signal cross-section for this process is 5.4 picobarn for 14 TeV p-p collisions**. We can assume that **the filter efficiencyin the signal samples was 0.17** for selecting diphoton events. Each signal sample contains **10,000 such simulated events**.

- Let's pretend that thediphoton background was instead generated using the Sherpa event generator, which takes care of both matrix-element and hadronization. The ** background cross-section for this process at 14 TeV was **12865 picobarn** with a **filter efficiency of 0.081**. The background sample contains **100,000 such simulated events**.

- we can pretend that both types simulation are overlaid with addition simulated crossing events to mimic the presence of additional interactions in the same collision event (known as pileup). Similarly, let's say that the interaction of the simulated particles with the ATLAS detector are modelled with the GEANT 9 programme.

- The data were collected within ATLAS during pp collision events at 14 TeV, during the years 2031-2043, during which time an integrated luminosity of **1298.1 /fb of data was accumulated**.

In order to avoid biasing yourself and designing your analysis around possible statistical fluctuations, you will be performing a “blind analysis”. This means that the observed data (which will be in the exact same format as the other two types of dataset), will not be made available to you until the very end of the day. You should make all your analysis decisions before you actually look at the data! The history of particle physics is littered with false discoveries by people who took shortcuts by peeking at the data, or changing their analysis after looking at the data. We will not fall into that trap. But let’s start to get familiar with the samples you have at your disposal now...


## First contact with the datasets

On the indico page for the course, you will find several files called `signal_xxx.csv` which contain each 10,000 simulated events for different assumptions of the X-boson mass. Each line (aside from the headers) corresponds to one event. Each columns represents one property. There is also a `background.csv` file in the same format with 100,000 simulated backgroudn events.

**TASK 1.1** First of all, let's load the data from the csv files into tables that we can easily manipulate.

Start by doing an `import` of the pandas library via `import pandas as pd` and read in the different datasets. What sort of information do you have in the dataset? Inspect it using the `Dataframe.head()` method.

In [ ]:
import pandas as pd
# ...

**TASK 1.2** Next, make sure you know how to manipulate `pandas Dataframe`. Can you:
- select the information for one column (for example, just the "photon1_pT")?
- select the information for one row (form example, the 111th row)?
- apply a mask/selection based on some of the variables (for example, pick only the dataframe rows where photon1_pT > 250 GeV)? How many entries are in the dataframe after such a cut?

You can use google to find examples if you need.

In [ ]:
# one column


In [ ]:
# one row


In [ ]:
# select just rows where photon1_pT > 150 GeV


**TASK 1.3:** Now that we can read in a file and access the observables from each event, we want to start making histograms, showing the distribution of each observable for each sample type. What we want is to draw several sample histograms on the same plot.

It may be worth defining a function to do this plotting step for you, taking as input the name of an observable and plotting histograms for the different datasets. A skeleton is already provided, but the plotting itself is missing...

- Add the lines that would make a histogram of the observable, making sure that it’s normalised to unit area.
- You can use `matplotlib.pyplot`’s `plt.hist()` function for this, with the `density=1` keyword for the unit normalisation. You'll need to import it `import matplotlib.pyplot as plt`
- To make sure the plots use the same binning for all the datasets, we can use the fact that `plt.hist()` returns the binning. You can catch it like this and use it for the next plot:
```
values, bins, other_stuff = plt.hist(data1, density=1, label="blah1")
plt.hist(data2, bins=bins density=1, label="blah2")
```
- Note that you can also use the `alpha=0.5` argument to make the plots slightly transparent.

Using the function, start by visualising how the `photon1_pT` compares between the different datasets.

In [ ]:
# function definition
import matplotlib.pyplot as plt
def plot_distribution(observable):

     _, bins, _ = plt.hist(df_bkg[observable], alpha=0.4, histtype='barstacked', stacked=False, density=1, label="Background")
     plt.hist(df_sig500[observable],bins=bins, alpha=0.4, histtype='barstacked', stacked=False, density=1, label="$m_X=500$ GeV")
     plt.hist(df_sig750[observable], alpha=0.4, bins=bins, histtype='barstacked', stacked=False, density=1, label="$m_X=750$ GeV")
     plt.hist(df_sig1000[observable], alpha=0.4,bins=bins, histtype='barstacked', stacked=False, density=1, label="$m_X=1000$ GeV")
     plt.legend()
     plt.xlabel(observable)
     plt.ylabel("a.u.")

In [ ]:
# call the function to see the distribution of the observable photon1_pT


Now let's use a loop to make plots for all variables. For this, make sure to add `%matplotlib inline` to the cell and `plt.show()` inside the loop and after your function.

Hint: You can obtain the column names to loop over with `df.columns`.

In [ ]:
%matplotlib inline
# hint: do something like this
# for colName in df.columns:
#    plotDistribution(colName)
#    plt.show()

Before we move to the next section, it's worth inspecting these various variables and seeing if we can already spot those where the distribution from signal and background is different. Indeed, some variables look the same for signal and background, others show some differences. This means we can apply *selections* (aka *cuts*) where we can remove background while retaining a high propotion of signal... and thus improve our sensitivity to the prospective X boson!

# Part 2: Invariant mass ...and normalisation

If the X decays to pairs of photons, then we expect a resonance in the di-photon invariant mass spectrum. In other words, the invariant mass of the two-photon system should be the same as the invariant mass of the X-boson... which ought to be distributed approximately like a Gaussian whose mean is the mass of the boson. The invariant mass spectrum of the background should not be distributed in such a way, but is more likely to be some kind of falling spectrum. So, let’s calculate the invariant mass and plot that too. this will become our discriminating variable. The formula relating invariant mass of a two-body system to its transverse momenta ($p_T$), pseudorapidity (η) and azimuthal angle (φ) is:

$$m_{inv} = \sqrt{2 p^T_1 p^T_2 (\cosh(η_1 − η_2) − \cos(φ_1 − φ_2))} . $$

**TASK 2.1:** Using the provided formula, add a column into each dataframe for the invariant mass of the two-photon system (using vectorized operations on the df columns!). Plot the distribution of that for signal and background as you did for the other variables

Hint: `numpy` has functions for cosh and cos. You may want to make a function to avoid repeating yourself. You can import it via `import numpy as np`.

In [ ]:
def add_minv(df):
    # Fill in here!
    return df

In [ ]:
# simething like this?
#df_sig500 = add_minv(df_sig500)
#...
#df_bkg = add_minv(df_bkg)

In [ ]:
# plot invariant mass

You should have found that the background invariant mass distribution is a falling spectrum, while the signals have gaussian invariant mass distributions centred around their hypothetical mass

We now would like to plot predicted event counts instead of distributions normalised to 1.
So what we would like is that the integral of each of the invariant mass plots should correspond to the predicted number of events at the LHC. The general formula to get the number of expected events in a sample is:
$$N_{exp}=L×σ×ε× \frac{N_{sel}}{ N_{gen}}$$

where $N_{exp}$ is the expected number of events in collisions, $L$ is the integrated luminosity of the collected dataset, $σ$ is the cross-section for the process under consideration, $ε$ is the filter-efficiency (if we restrict the event generation for example to a particular decay type of the particle, $X$ → γγ),  $N_{sel}$ is the number of selected events in your analysis, and $N_{gen}$ is the number of events generated, before any other selections.
In your case, to calculate individual event weights, you can take $N_{sel} = 1$, and $N_{gen}$ is the total number of events in the samples you have been given. **The required values for each dataset are given in the preamble**

**TASK 2.2** Calculate the normalisation weight for each sample, add at it as an extra column in each dataset, named `'w'` or `'weight'`. Then make a new function called `plot_weighted_distribution` which plots the sample plots as above, but where the samples are normalised using the weight you just calculated. You can remove the `density=1` keyword and replace it with the `weights=...` keyword.

In [ ]:
# example... df_bkg["w"] = 1298.1 * 5.4 * 0.17 * 1 / len(df_bkg)


In [ ]:
# hint : you can amend the code below to work with your notebook!
def plot_weighted_distribution(observable):

     _, bins, _ = plt.hist(df_bkg[observable], alpha=0.4, histtype='barstacked', stacked=False, bins=100, weights=df_bkg.w, label="Background")
     plt.hist(df_sig500[observable],bins=bins, alpha=0.4, histtype='barstacked', stacked=False,weights=df_sig500.w, label="$m_X=500$ GeV")
     plt.hist(df_sig750[observable], alpha=0.4, bins=bins, histtype='barstacked', stacked=False, weights=df_sig750.w,label="$m_X=750$ GeV")
     plt.hist(df_sig1000[observable], alpha=0.4,bins=bins, histtype='barstacked', stacked=False, weights=df_sig1000.w,label="$m_X=1000$ GeV")
     plt.legend()
     plt.xlabel(observable)
     plt.ylabel("a.u.")

In [ ]:
# plot invariant mass after normalisation

You should now be able to see a small signal which is well inside the background. How can we make it more visible?

# Part 3: Designing the selection

So far, you have become familiar with your datasets and learnt to make some plots to show the simulated distributions of signals and backgrounds in your search for the X boson. But as we saw, the correctly-normalised signal is tiny compared to the background, and without further effort it’s unlikely that we’d be able to distinguish it from the background within the uncertainties. So, we need to try to make our analysis more sensitive! Now, in the “true” dataset, the background may not be perfectly modelled, and the signal (if it is there!) is unlikely to look exactly like the ones you are practicing with. But you can assume that these simulated samples are close enough to what you will encounter in the “observed” data to be able to design a selection which is optimised for your analysis.
The first thing we need to do is try to get a realistic estimate of the background and signal we might expect to find. Remember that the X boson is only expected to exist in a given range of possible masses, between 500 and 1000 GeV. So we can already define a first search region (SR) where we will be looking for our signal. To have a safety margin, we can add a first selection (or in the jargon a cut ):


The first thing we need to do is try to get a realistic estimate of the background and signal we might expect to find. Remember that the X boson is only expected to exist in a given range of possible masses, between 500 and 1000 GeV. So we can already define a first search region (SR) where we will be looking for our signal. To have a safety margin, we can add a first selection (or in the jargon a "cut" ):

$$m_{inv} ∈ [400, 1100] \text{ GeV}$$

**TASK 3.1:** Count the (weighted) number of background events predicted in the background simulation in your basic SR by applying the invariant mass cut and taking the sum of weights of the events which survive. Do the same for one of your signal samples. You can then calculate your sensitivity $s$ as
$$ s = \frac{\sum w_{\text{signal}}}{\sqrt{\sum w_\text{bkg}}} $$

How many entries survived the cut?

In [ ]:
# hint: first, use the new "minv" variable you made in task 2.1
# to define a mask to select only masses within 400-1100 GeV.
# Make a copy of your background dataframe and apply the mask to it.
# Do the same for your signal sample(s).
# extra hint: you can use np.logical_and(mask1, mask2) to make a combination of masks,
# for example minv > 400 and minv < 1100

# somthing like...
# mask_bkg = np.logical_and(df_bkg["minv"] < 1100 , df_bkg["minv"] > 400)
# df_bkg_minv = df_bkg[mask_bkg]

In [ ]:
# hint: create a function that takes as input the background and signal dataframes
# and returns the sensitivity. To get the sum of weights in your new reduced dataframe, use
# np.sum(df['weight']), where 'weight' is the column you added in Task 2.2
def get_sensitivity(df_s, df_b):
    # s = ...
    return s

In [ ]:
# calculate the sensitivity for each one of the signal datasets, you should get around 3


$s$ is what we call a *figure of merit*, the higher it is, the better is our analysis and the more chance we have to discover a new particle if it's there, or the stronger the limit we can set if we see no new particle. The `sqrt` is there because `sqrt(bkg)` corresponds roughly to the *uncertainty* on the background estimate. So this $s$ is effectively telling you how large your predicted signal is compared to your background.

What we want to do now is try to apply additional selections to *maximise* $s$ as much as possible. In other words, we want to get the most signal we can while removing the most background.  In general we will not really want to touch any of the variables which might contribute to the `m_inv` calculation (we want to avoid biasing this spectrum for reasons which we will see later). But you have several other variables which are a priori uncorrelated with the invariant mass, and where we could get some extra sensitivity. For example, if you look back at the distribution of the `n_jets`, you see that there are clearly regions where we can add a cut which would remove background without really removing any signal, which would obviously improve our $s$!

**TASK 3.2:** Try to define a new mask which excludes some parts of the `n_jets` spectrum, where there is more background than signal. For example `n_jets < 5` (check this on your plots!).
Use that to make new dataframes with this new selection applied alongside your "minv" mass. What are your new values of $s$ after applying this selection ?
I got $s=4.4$ or so

In [ ]:
# hint, you can maybe define a new mask using np.logical_and(mask_SR, df['n_jets'] < 5) and apply it to your original dataset.
# alternatively, you can just apply the same process as you did in Task 3.1 but on your already-reduced dataset.
#
# it could help to define a function for this sort of thing, if you like
def apply_selection(df, variable, cutValue = 0, upperOrLower= ">"):
    if upperOrLower=="<":
        mask= df[variable] < cutValue
    elif upperOrLower==">":
        mask= df[variable] > cutValue
    else:
        print(f'{upperOrLower} not a defined operation!!')
    return df[mask]

# and call it something like this:
# cutVal=5
# df_bkg_SR_after_cuts = apply_selection (df_bkg_SR , "n_jets" , cutVal, "<")

In [ ]:
# get sensitivity after applying the selection on the number of jets


Try different values of the `n_jets` cut (like 1, 2, 3, 4, 5, 6, 7, 8, 9...) to see which of them gives you the best values of $s$.
The best value I got was $s=5.6$.

In [ ]:
# loop over integer values between 1 and 10 and print the sensivity for each njet choice
# hint: here is how I did it, can you do something similar?
# for nj in range(1,9):
#     df_bkg_SR_after_cuts = apply_selection (df_bkg_SR , "n_jets" , nj, "<")
#     df_sig500_SR_after_cuts = apply_selection (df_sig500_SR , "n_jets" , nj, "<")
#     s = get_sensitivity(df_sig500_SR_after_cuts, df_bkg_SR_after_cuts)
#     print(f"For njets > {nj}, we get sensitivity {s:.2f}")

**TASK 3.3:** Now that you know how to apply a cut, you can look at the other variables and apply additional selections for those, combining the selections on all the variables to get your final selection.

The variables you should consider are
- n_jets
- n_leptons
- photon1_isolation
- photon2_isolation
  
(but of course adding in the others may add extra sensitivity)

The simplest thing to do is to look at the plots you made above and estimate what selections to make by eye, calculate the new $s$, and manually play around with the selections to see when you reach a maximum.

If you are feeling more ambitious you could write a function which automatically scans the range of possible cuts to get the one with higest $s$ for each variables.

If you are feeling REALLY ambitious you could train a simple machine learning algorithm to simulatenously epxloit all these variables in the most optimal way :)

In [ ]:
# hint: this is how I did it... perhaps you can do something similar?

# njetCut = ???
# nLeptonsCut = ???
# photon1_isolationCut = ???
# photon2_isolationCut = ???

# df_bkg_selection = apply_selection (df_bkg_SR , "n_jets" , njetCut, "<")
# df_bkg_selection = apply_selection (df_bkg_selection , "n_leptons" , nLeptonsCut, ">")
# df_bkg_selection = apply_selection (df_bkg_selection , "photon1_isolation" , photon1_isolationCut, "<")
# df_bkg_selection = apply_selection (df_bkg_selection , "photon2_isolation" , photon2_isolationCut, "<")

# df_sig500_selection = apply_selection (df_sig500_SR , "n_jets" , njetCut, "<")
# df_sig500_selection = apply_selection (df_sig500_selection , "n_leptons" , nLeptonsCut, ">")
# df_sig500_selection = apply_selection (df_sig500_selection , "photon1_isolation" , photon1_isolationCut, "<")
# df_sig500_selection = apply_selection (df_sig500_selection , "photon2_isolation" , photon2_isolationCut, "<")

# s = get_sensitivity(df_sig500_selection, df_bkg_selection)
# print(f"Sensitivity is {s:.2f}")

I was able to get a $s$ around 7 using the four basic variables listed above and optimising "by hand".

**TASK 3.4** Now that you have defined your set of selections, apply this to your original dataframe (without the $m_{inv}$ window cut) and re-make your plot of the invariant mass spectrum.
Does this look promising in terms of a potential discovery ?

# Part 4: Statistical analysis

Now that you have defined your selection, you need to perform the statistical analysis: estimating the amount of background in your signal region, and determining what you will do if you see an excess of events above your estimate: measuring the significance and the mass of the X boson if you have an excess... and setting limits on the cross-section if not!

The first step will be the background estimate: your background simulation is not guaranteed to be a perfect description of your data. If was fine to derive your selection, but if there are small differences in the simulation and what nature serves you in the distribution of the background invariant mass spectrum, then you might over- or under-estimate your prediction in the SR, leading potentially to a false discovery claim or a missed discovery!
How do we deal with this possibility? The best way forward is to use a so- called data-driven background estimate. In a nutshell, you can use the events which pass into “sideband” region which is similar to your SR but inverting the m_inv window, to estimate the background. Practically speaking, this means to parameterise the shape of the falling invariant mass spectrum using the events in side- band), where you know there is no signal. Then use the parametrised function to guess the background contribution in the SR (and it’s uncertainty!). Then when we look at the data in that region, we can check if there is an excess of events compared to what is predicted. Let’s test this procedure first on your background simulation.

**TASK 4.1:** Using your simulated background events, make a plot of the sideband of the `m_inv` spectrum of your SR: in other words, plot events passing all the cuts in your selection, but inverting the invariant mass cut:
$$m_\text{inv} \notin [400, 1100] \text{ GeV}$$

In [ ]:
# hint: you can amend the code below to work with your code
# do you understand what each step is doing?

# mask_sideband_bkg = np.logical_or(df_bkg_selection["minv"]< 400, df_bkg_selection["minv"]> 1100)
# df_bkg_sideband = df_bkg_selection[mask_sideband_bkg]
# df_bkg_sr = df_bkg_selection[~mask_sideband_bkg]

# n, bins, _ = plt.hist(df_bkg_sideband.minv, bins=100, weights=df_bkg_sideband.w)

It looks like this shape could be fitted by an exponential...
Parametrising a distribution means fitting a function to the shape. A good option for this is to use the `scipy.optimize.curve_fit` package. If you import the `helpers.py` (written by me tpo simplify this tutorial) package, you fill find a convenient wrapper do fit, some predefined functions (you can add your own too!) and a helper to work out the post-fit yield in a given range.

In [ ]:
!ls SearchingTheXBoson | grep help

helpers.py


We are also going to need to install the `pyhf` package, a useful library to help perform the statistical analysis.

In [ ]:
!pip install pyhf

**TASK 4.2:** Import the `helpers.py` package.
Adjust the code in the hint below to fit your sidebands to an exponential function using the provided fitting tool (which is using `scipy.curve_fit` under thehood).

Again, by amending the code in the hint, determine the estimated background yield in the SR: effectively, the integral of this function over the SR’s invariant mass window  [400, 1100] GeV. This is your **background estimate**.

In [ ]:
# hint: you can amend the code below to work with your code
# do you understand what each step is doing?

# import SearchingTheXBoson.helpers as hp
# n, bins, _ = plt.hist(df_bkg_sideband.minv, bins=np.linspace(50,2000, 40), weights=df_bkg_sideband.w)
# plt.hist(df_bkg_sr.minv, bins=np.linspace(50,2000, 40), weights=df_bkg_sr.w)
# x, params, errs = hp.do_fit(hp.Expo, bins, n)

# plt.plot(x, hp.Expo(x, *params), label = "Expo fit")
# bkgest, bkgest_err =  hp.integral_with_errors(hp.Expo, x, (400,1100), params, errs)
# print (f"Our estimated background yield  is {bkgest:.2f} +/- {bkgest_err:.2f}")

### Determining if you see an excess

In the final session, we will repeat the background estimate procedure with the “observed” data: the parametrisation will be a little different but the estimate ought to be done in the same way. Then you will have two numbers: first, the estimated number of events in the SR from the sideband fit (your expected background), and the uncertainty on that number; second, the actual number of events you count in the SR. What you are trying to do is determine if you see an excess of observed events compared to the prediction. The prediction is effectively your null hypothesis (there is no X boson). You are trying to see if the data you observe are closer to the alternative hypothesis that the X boson exists. We define the significance

$$Z = (N_\text{predicted} − N_{\text{observed}})/ \text{err}_\text{predicted},$$

where $\text{err}_\text{predicted}$ is the uncertainty on your predicted number of events. If you see an “excess” of events, we can try to gauge how likely this was to have come from a statistical fluctuation using the significance.

A significance of 1 (aka “1-σ”) is usually consistent with a statistical fluctuation.
A 3-σ detection has a 0.3% probability of occurring by chance, and 5-σ is just a 0.00006% probability of occurring by chance. Physicists traditionally call a 3-σ detection “evidence”, while a 5-σ detection is considered a “discovery”...
Let’s continue to practice with the simulated data to rehearse what might happen next week.

**TASK 4.3:** Start by checking the SR using the dataset composed only of your background simulation. In other words, evaluate the sum of weights for events which are actually in the SR (not the ones from sideband estimate, but whose which really pass the selection and with `m_inv` $\in [400, 1100]$ GeV)... and evaluate the significance.

In principle, the observed and expected number of events should agree in this case, if your parametrisation was good! And therefore the significance should be low (since there is no X boson in this dataset, and no excess!)

In [ ]:
# Hint: you can amend the code below to work with your code
# do you understand what each step is doing?

# df_bkg_SR = df_bkg_selection[~mask_sideband_bkg]
# n_obs = np.sum(df_bkg_SR.w)

# Z = (n_obs - bkgest)/ bkgest_err

# print (f" Nobs = {n_obs:.2f}, Npred = {bkgest:.2f}+/-{bkgest_err:.2f}, Z = {Z:.2f}σ")

**TASK 4.4:** Repeat the exercise, but this time, counting the sum of eights of events in the SR with the background and one of your signal samples (pandas.concat is useful to add dataframes together). Do the expected and observed number
of events agree this time? If not, what is the significance? If this happens with your "real" dataset, will you be able to claim a discovery ?



In [ ]:
# Hint: you can amend the code below to work with your code
# do you understand what each step is doing?

# mask_SR_sig500 = np.logical_and(df_sig500_selection["minv"]> 400, df_sig500_selection["minv"]< 1100)
# df_sig500_SR = df_sig500_selection[mask_SR_sig500]

# df_sigBkg_SR = pd.concat([df_sig500_SR, df_bkg_SR])

# n_obs = np.sum(df_sigBkg_SR.w)

# Z = (n_obs - bkgest)/ bkgest_err

# print (f" Nobs = {n_obs:.2f}, Npred = {bkgest:.2f}+/-{bkgest_err:.2f}, Z = {Z:.2f}σ")

## If you see an excess (3-σ or more)... and what to do if not.

**If you see an excess** in your data which is statistically significant (> 3σ), you will need to quantify it in terms of its significance. People will want to know the mass of the boson you claim to have discovered. To do that you can plot the `m_inv` distribution. You should be able to see a bump on your spectrum at the mass of the discovered boson! Thee size of the bump will depend on the cross-section. One would determine those precisely with a fit.

**If you do not see an excess** in your data, you will want to set a limit on the maximum allowed cross-section (if the cross-section was large, you'd have seen it, so it must be smaller than a certain value). This is done with the CLs method.

# Part 5: Try it on your own dataset!

Now you will be assigned a number, which corresponds to an unique dataset in which there may be hidden an X boson of a randomly-chosen mass and cross-section.

Your job is not to:
- read your dataset into a dataframe
- apply your selection
- plot the invariant mass spectrum
- do a fit of the sidebands and determine the background estimate in the SR (and its uncertainty).
- compare it to the observed number of events you observed and determine the significance.

NB: the data does not need any normalisation weights! (ie all weights=1)

Before we start, let's check if you have all the experimental datasets available:

In [ ]:
!ls SearchingTheXBoson/datasets_experiment

data_0.csv  data_2.csv	data_4.csv  data_6.csv	data_8.csv
data_1.csv  data_3.csv	data_5.csv  data_7.csv	data_9.csv


Now, let's generate a random number for you! Replace the seed below with your favourite number and run the cell.

In [13]:
seed = ???

import numpy as np
rng = np.random.default_rng(seed)
number = rng.integers(0,10)

print(f"You got dataset data_{number}.csv")

**Task 5.1:** Perform the data analysis in your dataset and get the significance. Did you make a discovery or refute Dr. X's theory ?

In [ ]:
## Determine your signifiance below!

# open the dataset and add invariant mass variable

# apply the selection

# define the sideband and SR

# do the fit like before, maybe something along these lines?

# n, bins, _ = plt.hist(df_data_sideband.minv, bins=np.linspace(50,2000, 40))
# plt.hist(df_data_SR.minv, bins=np.linspace(50,2000, 40))
# x, params, errs = hp.do_fit(hp.Expo, bins, n)
# hp.fixedExpoN, hp.fixedExpoL = params[0], params[1]
# plt.plot(x, hp.Expo(x, *params), label = "Expo fit")
# bkgest, bkgest_err =  hp.integral_with_errors(hp.Expo, x, (400,1100), params, errs)
# print (f"Our estimated background yield  is {bkgest:.2f} +/- {bkgest_err:.2f}")

# get the observed yield by counting data events

# n_obs = len(df_data_SR)
# Z = (n_obs - bkgest)/ bkgest_err
# print (f"Nobs = {n_obs:.2f}, Npred = {bkgest:.2f}+/-{bkgest_err:.2f}, Z = {Z:.2f}σ")

(If you made a discovery) **Task 5.2a:** What is the mass of the discovered boson, and it's cross-section?

In [ ]:
## hint: uncomment the code below and amend it to work in your code

# n, bins, _ = plt.hist(df_data_selection.minv, bins=np.linspace(50,2000, 40))

# x, params, errs = hp.do_fit(hp.GausPlusFixedExpo, bins, n)
# plt.plot(x, hp.GausPlusFixedExpo(x, *params), label = "Expo plus Gaussian fit")
# y, ye = hp.integral_with_errors(hp.Gaussian, x, (400,1100), params[:3], errs[:3])

# xsFactor = 5.4/sum(df_sig500_selection.w) # this depends on your own cuts
# print(params, hp.fixedExpoN, hp.fixedExpoL)
# print("measured yield: %.2f +/- %.2f " % ( y, ye))
# print("measured cross-section: %.2f +/- %.2f pb" % (  y*xsFactor, ye*xsFactor))
# print("measured mass: %.2f +/- %.2f GeV" % ( params[0], errs[0]))


(If you saw no excess) **Task 5.2b:** What are the limits according to the CLs method?


In [ ]:
## hint: uncomment the code below to determine the excluded region

# signal_pred = sum(df_sig500_selection.w)
# x = hp.cls_limit_calculator([signal_pred], [bkgest], [bkgest_err], [n_obs])